## 1.2 Skapa gles matris

In [13]:
#1 Reading in the csv-files
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from fuzzywuzzy import process
movieData= pd.read_csv('./Data/movies.csv', usecols=["movieId", 'title'], dtype={'movieId': 'int32', 'title': 'str'})
ratingData = pd.read_csv('./Data/ratings.csv', usecols=["userId", "movieId", "rating"], dtype ={'userId': 'int32', 'movieId': 'int32', 'rating': 'float32'})
ratingData.head()

,userId,movieId,rating
0,1,307,3.5
1,1,481,3.5
2,1,1091,1.5
3,1,1257,4.5
4,1,1449,4.5


In [14]:
# Sorting out all movies with few ratings and only saving top 3500 movies with most ratings.
movies_with_few_ratings = ratingData["movieId"].value_counts().sort_values(ascending=False).head(3500)
new_ratingData = ratingData[ratingData['movieId'].isin(movies_with_few_ratings)]
len(new_ratingData)


5565007

In [15]:
#Getting rid of some more data, getting rid of movies with below 4 in rating.
sorted_ratingData = new_ratingData.drop(new_ratingData[new_ratingData.rating < 4].index)
len(sorted_ratingData)

2752824

In [16]:
#Creating the matrix
movies_users=sorted_ratingData.pivot(index='movieId', columns='userId', values='rating').fillna(0)
mat_movies_users=csr_matrix(movies_users.values)

In [17]:
# training the model
model_knn=NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20)
model_knn.fit(mat_movies_users)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=20)

## 1.3 Rekommenderarsystemet

In [18]:
# Creating the model to predict and predicting the "Shawshank redemption"
def recommender(movie_name, data, model, n_recommendations):
    model.fit(data)
    idx=process.extractOne(movie_name, movieData['title'])[2]
    print("Movie Selected: ", movieData['title'][idx], "Index: ",idx)
    print("Searching for recommendations...")
    distances, indices = model.kneighbors(data[idx], n_neighbors=n_recommendations+1)
    print(movieData.loc[indices[0][1:]])

recommender('Shawshank redemption', mat_movies_users, model_knn, 5)

Movie Selected:  Shawshank Redemption, The (1994) Index:  315
Searching for recommendations...
     movieId                                              title
348      352                                    Crooklyn (1994)
289      292                                    Outbreak (1995)
413      417                                   Barcelona (1994)
332      336                           Walking Dead, The (1995)
290      293  Léon: The Professional (a.k.a. The Professiona...


## Recommendation system 1.3 questions

- a) Systemet hittar de närmsta datapunkterna från olika users ratings för att rekommendera nästa film, i ett flerdimensionellt system. I min modell kollar den på de 20 närmaste punkterna. Den tar alltså och kollar på hur andra personer som ratat filmen likadant och försöker hitta punkter från andra filmer som ligger likadant.

- b) Bra rekommenderar system använder oftast olika modeller kombinerat. T.ex använder de modeller som kollar på din egen filmhistorik för att hitta filmer som liknar de filmer du sett, utifrån rating, genre, etc men oftast är det kombinerat med ett system som ovan där den också kollar på olika "user profiles" och matcher de filmer du har sett med liknande users och därmed kan rekommendera utifrån hur andra användare som liknar dig kollar på för fler filmer.
